In [2]:
import pandas as pd
import numpy as np

# read in the book and user datasets
books = pd.read_csv('../data/books_v2.csv')
users = pd.read_csv('../data/users.csv')

# merge the datasets on a common field
merged_data = pd.merge(books_and_users, on='location')

# randomly assign a user to each book
merged_data['userId'] = np.random.choice(merged_data['userId'], size=len(merged_data))

# duplicate the location and Occupied columns to the book dataset
merged_data['location'] = merged_data['location_y']
merged_data['Occupied'] = merged_data['Occupied_y']

# drop the redundant location and Occupied columns
merged_data.drop(['location_x', 'location_y', 'Occupied_x', 'Occupied_y'], axis=1, inplace=True)

# save the final dataset to a CSV file
merged_data.to_csv('../data/books_with_users.csv', index=False)


NameError: name 'books_and_users' is not defined

In [3]:
import pandas as pd
import geopy.distance
import tensorflow as tf
from keras.models import load_model
import tokanizer_and_cleaner
import numpy as np

# Load the dataset
df = pd.read_csv('../data/books_with_users.csv')

# Preprocessing and data cleaning
df = df[['book_id', 'book_authors', 'book_desc', 'book_format', 'book_isbn', 'book_pages', 'book_rating', 'book_rating_count', 'book_review_count', 'book_title', 'genres', 'genre_count', 'genre_list', 'label', 'userId', 'bookId', 'location']]
df['book_authors'] = df['book_authors'].apply(lambda x: x.split('|'))
df = df.dropna()

# Load the LSTM model
lstm_model = load_model('../models/BookModel1.h5')

# Define a function to predict book genre using the LSTM model
def predict_genre(description):
    # # Tokenize the input description
    # tokenizer = tf.keras.preprocessing.text.Tokenizer()
    # tokenizer.fit_on_texts(df['book_desc'])
    # seq = tokenizer.texts_to_sequences([description])
    # padded_seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=1000)
    # # Make the prediction
    # pred = lstm_model.predict(padded_seq)[0][0]
    # # Map the prediction to a genre label
    # genre_label = 'fiction' if pred > 0.5 else 'non-fiction'
    max_desc_length=200

    labels = ['fiction', 'nonfiction']
    cleaned_text = tokanizer_and_cleaner.clean_text(description)
    tokenized_text = tokanizer_and_cleaner.tokenizer(cleaned_text, tokanizer_and_cleaner.vocab_dict, max_desc_length)
    input_data = np.reshape(tokenized_text, (1, max_desc_length))
    output = lstm_model.predict(input_data, batch_size=1)
    score = (output > 0.5) * 1
    pred = score.item()
    return labels[pred]

# Define a function to compute distances using the user's location for geographical recommendation
def compute_distance(row, location):
    loc1 = eval(row['location'])
    loc2 = location
    return geopy.distance.geodesic(loc1, loc2).km

# Define rules for rule-based recommendation
def rule_based_recommendation(row, description):
    if description in row['book_title'] or description in row['book_authors']:
        return True
    elif row['book_rating'] >= 4.0:
        return True
    else:
        return False

# Define a function to generate hybrid recommendations
def hybrid_recommendation(description, location, num_recommendations):
    # Compute geographical distances for all items
    distances = df.apply(compute_distance, axis=1, args=(location,))

    # Compute rule-based scores for all items
    rule_based_scores = df.apply(rule_based_recommendation, axis=1, args=(description,))

    # Compute LSTM-based scores for all items
    lstm_scores = df['book_desc'].apply(predict_genre).apply(lambda x: 1 if x == 'fiction' else 0)

    # Combine the different recommendation scores into a weighted score
    weights = [0.3, 0.3, 0.4]
    scores = weights[0] * (1 / (distances + 1)) + weights[1] * rule_based_scores + weights[2] * lstm_scores
    df['hybrid_score'] = scores

    # Sort by hybrid score and recommend the top items
    top_items = df.sort_values('hybrid_score', ascending=False).head(num_recommendations)

    return top_items[['book_title', 'book_authors', 'book_rating', 'location']]


In [4]:
description = "A great book about history"
location = [7.000554153055612, 79.93576326258663]
num_recommendations = 10

recommendations = hybrid_recommendation(description, location, num_recommendations)
print(recommendations)

1/1 [==============================] - 0s 43ms/step


KeyboardInterrupt: 

In [6]:

import pandas as pd
import geopy.distance
import tensorflow as tf
from keras.models import load_model
import tokanizer_and_cleaner
import numpy as np

# Step 1: Load the user_book.csv dataset
df = pd.read_csv('../data/books_with_users.csv')

# Step 2: Load the saved LSTM model
model = load_model('../models/BookModel1.h5')

# # Step 3: Preprocess the text data in the description column of the df dataframe
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df['description'])
#
# maxlen = 100 # maximum sequence length
# X = tokenizer.texts_to_sequences(df['description'])
# X = pad_sequences(X, maxlen=maxlen)
#
# # Step 4: Use the loaded LSTM model to predict the genre of books
# y_pred = model.predict(X)
# predicted_genre = ['fiction' if p >= 0.5 else 'non-fiction' for p in y_pred]
max_desc_length=200

labels = ['fiction', 'nonfiction']
cleaned_text = tokanizer_and_cleaner.clean_text(description)
tokenized_text = tokanizer_and_cleaner.tokenizer(cleaned_text, tokanizer_and_cleaner.vocab_dict, max_desc_length)
input_data = np.reshape(tokenized_text, (1, max_desc_length))
output = lstm_model.predict(input_data, batch_size=1)
score = (output > 0.5) * 1
pred = score.item()


# Step 5: Compare the predicted genre with the actual genre in the label column of the df dataframe
df['predicted_genre'] = labels[pred]

# Step 6: Filter the top 10 books with the predicted genre
top10_books = df[df['predicted_genre'] == 'fiction'].head(10)

# Step 7: Print the top 10 books with the predicted genre
print(top10_books)


1/1 [==============================] - 0s 69ms/step
   book_id                book_authors   
0        0             Suzanne Collins  \
1        1  J.K. Rowling|Mary GrandPré   
2        2                  Harper Lee   
3        3             Stephenie Meyer   
4        4                Markus Zusak   
5        5   C.S. Lewis|Pauline Baynes   
6        6           Margaret Mitchell   
7        7                  John Green   
8        8               Douglas Adams   
9        9            Shel Silverstein   

                                           book_desc   
0  Winning will make you famous. Losing means cer...  \
1  There is a door at the end of a silent corrido...   
2  The unforgettable novel of a childhood in a sl...   
3  About three things I was absolutely positive.F...   
4  Trying to make sense of the horrors of World W...   
5  Journeys to the end of the world, fantastic cr...   
6  Gone with the Wind is a novel written by Marga...   
7  Despite the tumor-shrinking medica

In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load the dataset
df = pd.read_csv('books_data.csv')

# Define the rule-based recommendation function
def rule_based_recommendation(df, occupied):
    if occupied == 1:
        # If the user is currently occupying a location, recommend books with high ratings
        recommendations = df[df['book_rating'] >= 4.0].sort_values(by=['book_rating'], ascending=False)
    else:
        # If the user is not currently occupying a location, recommend popular books
        recommendations = df.sort_values(by=['book_rating_count', 'book_review_count'], ascending=False)

    # Return the top 10 recommendations
    return recommendations.head(10)

# Define the geographical recommendation function
def geographical_recommendation(df, location):
    # Calculate the distance between the user's location and each book's location
    df['distance'] = df['location'].apply(lambda x: geodesic(location, eval(x)).km)

    # Sort the books by distance
    recommendations = df.sort_values(by=['distance'])

    # Return the top 10 recommendations
    return recommendations.head(10)

# Define the hybrid recommendation function
def hybrid_recommendation(df, occupied, location):
    # Get the recommendations from the rule-based and geographical recommendation systems
    rule_based_recommendations = rule_based_recommendation(df, occupied)
    geographical_recommendations = geographical_recommendation(df, location)

    # Combine the two sets of recommendations and remove duplicates
    recommendations = pd.concat([rule_based_recommendations, geographical_recommendations])
    recommendations = recommendations.drop_duplicates(subset='book_id')

    # Return the top 10 recommendations
    return recommendations.head(10)

# Example usage
location = [7.000554153055612, 79.93576326258663]
occupied = 1
recommendations = hybrid_recommendation(df, occupied, location)
print(recommendations[['book_title', 'book_rating', 'distance']])
